In [19]:
import numpy as np
import roboticstoolbox as rtb
from spatialmath import *
import pandas as pd
from math import pi
import matplotlib.pyplot as plt
from matplotlib import cm
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

%matplotlib notebook

The Toolbox supports models defined using a number of different conventions.  We will load a very classical model, a Puma560 robot defined in terms of standard Denavit-Hartenberg parameters

In [2]:
p560 = rtb.models.Z1()

Now we can display the simple Denavit-Hartenberg parameter model

In [3]:
print(p560)

ERobot: Z1_DESCRIPTION (by Unitree), 7 joints (RRRRRRR), dynamics, geometry, collision
┌─────┬───────────────┬───────┬───────────────┬───────────────────────────────┐
│link │     link      │ joint │    parent     │      ETS: parent to link      │
├─────┼───────────────┼───────┼───────────────┼───────────────────────────────┤
│   0 │ world         │       │ BASE          │                               │
│   1 │ link00        │       │ world         │ SE3()                         │
│   2 │ link01        │     0 │ link00        │ SE3(0, 0, 0.0585) ⊕ Rz(q0)    │
│   3 │ link02        │     1 │ link01        │ SE3(0, 0, 0.045) ⊕ Ry(q1)     │
│   4 │ link03        │     2 │ link02        │ SE3(-0.35, 0, 0) ⊕ Ry(q2)     │
│   5 │ link04        │     3 │ link03        │ SE3(0.218, 0, 0.057) ⊕ Ry(q3) │
│   6 │ link05        │     4 │ link04        │ SE3(0.07, 0, 0) ⊕ Rz(q4)      │
│   7 │ link06        │     5 │ link05        │ SE3(0.0492, 0, 0) ⊕ Rx(q5)    │
│   8 │ gripperStator │       │ l

The first table shows the kinematic parameters, and from the column titles we can see clearly that this is expressed in terms of standard Denavit-Hartenberg parameters.  The first column shows that the joint variables qi are rotations since they are in the θ column.  Joint limits are also shown.  Joint flip (motion in the opposite sense) would be indicated by the joint variable being shown as for example like `-q3`, and joint offsets by being shown as for example like `q2 + 45°`.

The second table shows some named joint configurations.  For example `p560.qr` is 

In [4]:
p560.qr

array([0.000922,  0.00068, -0.003654, -0.07501, -0.00013,  3.5e-05,        0])

In [5]:
p560.grab3
#cartesian space posture:  1.299 -0.485 -0.959  0.212 -0.142  0.674
#install with pip install -e .
#then you can add config in the Z1
#restart jupyter kernel after adding configs

array([  -0.247,    1.271,   -1.613,   -0.267,   -0.617,    0.916,        0])

If the robot had a base or tool transform they would be listed in this table also.

This object is a subclass of `DHRobot`, equivalent to the `SerialLink` class in the MATLAB version of the Toolbox.
This class has many methods and attributes, and we will explore some of them in this notebook.

We can easily display the robot graphically

In [6]:
p560.plot(p560.grab3);

where `qn` is one of the named configurations shown above, and has the robot positioned to work above a table top.  You can use the mouse to rotate the plot and view the robot from different directions.  The grey line is the _shadow_ which is a projection of the robot onto the xy-plane.

In this particular case the end-effector pose is given by the forward kinematics

In [7]:
p560.fkine(p560.grab3) #0.212 -0.142  0.674

   0.5072   -0.03828  -0.861     0.2117    
  -0.7246    0.5219   -0.4501   -0.1424    
   0.4666    0.8521    0.237     0.6744    
   0         0         0         1         


which is a 4x4 SE(3) matrix displayed in a color coded way with rotation matrix in red, translation vector in blue, and constant elements in grey.  This is an instance of an `SE3` object safely encapsulates the SE(3) matrix.  This class, and related ones, are implemented by the [Spatial Math Toolbox for Python](https://github.com/petercorke/spatialmath-python).

You can verify the end-effector position, the blue numbers are from top to bottom the x-, y- and z-coordinates of the end-effector position, match the plot shown above.

We can manually adjust the joint angles of this robot (click and drag the sliders) to see how the shape of the robot changes and how the end-effector pose changes

In [8]:
#p560.teach(); # works from console, hangs in Jupyter

An important problem in robotics is _inverse kinematics_, determining the joint angles to put the robot's end effector at a particular pose.

Suppose we want the end-effector to be at position (0.5, 0.2, 0.1) and to have its gripper pointing (its _approach vector_) in the x-direction, and its fingers one above the other so that its _orientation vector_ is parallel to the z-axis.

We can specify that pose by composing two SE(3) matrices:

1. a pure translation
2. a pure rotation defined in terms of the orientation and approach vectors

In [9]:
T = SE3(0.5, 0.2, 0.5) * SE3.OA([0,0,1], [1,0,0])
T

   0         0         1         0.5       
   1         0         0         0.2       
   0         1         0         0.5       
   0         0         0         1         


Now we can compute the joint angles that results in this pose

In [10]:
sol = p560.ikine_LM(T,q0=p560.qn,joint_limits=1) # give it initial from the end of previous traj

which returns the joint coordinates as well as solution status

In [11]:
sol

IKSolution(q=array([  0.1063,    2.367,   -2.849,   0.4826,    1.357,    1.571,   0.1074]), success=False, iterations=2454, searches=100, residual=3.7969755766788736e-07, reason='iteration and search limit reached, solution found but violates joint limits')

indicating, in this case, that there is no failure. The joint coordinates are

In [12]:
sol.q

array([  0.1063,    2.367,   -2.849,   0.4826,    1.357,    1.571,   0.1074])

and we can confirm that this is indeed an inverse kinematic solution by computing the forward kinematics

In [13]:
p560.fkine(sol.q)

  -1.101e-06  1.161e-06  1         0.5       
   1         1.454e-05  1.101e-06  0.2       
  -1.454e-05  1        -1.161e-06  0.5       
   0         0         0         1         


which matches the original transform.

A simple trajectory between two joint configuration is

In [14]:
q2 = sol.q

In [15]:
qt = rtb.tools.trajectory.jtraj(p560.qz, q2, 250) #250 is how many steps to generate, take eucledian distance to last traj point and some speed to calculate better

The result is a _namedtuple_ with attributes `q` containing the joint angles, as well as `qd`, `qdd` and `t` which hold the joint velocity, joint accelerations and time respectively.  

The joint angles are a matrix with one column per joint and one row per timestep, and time increasing with row number.

In [54]:
qt.q
id=np.arange(250)
# zeros=np.zeros(250)
np.hstack((id[:,None],qt.q,qt.qd)) #save this to traj file for Z1


array([[       0, -0.003791,   0.6372, ...,        0,        0,        0],
       [       1, -0.003791,   0.6372, ..., 0.0001105, -1.094e-06, 0.0002896],
       [       2, -0.003792,   0.6372, ..., 0.0004385, -4.342e-06, 0.001149],
       ...,
       [     247,   -0.247,    1.271, ..., 0.0004385, -4.342e-06, 0.001149],
       [     248,   -0.247,    1.271, ..., 0.0001105, -1.094e-06, 0.0002896],
       [     249,   -0.247,    1.271, ...,        0,        0,        0]])

We can plot this trajectory as a function of time using the convenience function `qplot`

In [17]:
# rtb.tools.trajectory.xplot(qt.q, block=False)

and then we can animate this

In [66]:
p560.plot(qt.q, dt=0.04);

In [40]:
file = np.genfromtxt('Traj_grab3.csv', dtype=float, delimiter=',')
first_traj = file[500:-2500, 1:8]
last_pos = first_traj[-1,:]
qt = rtb.tools.trajectory.jtraj(last_pos,
                                p560.grab3, 250)
final_traj = np.vstack((first_traj[:-1,:],qt.q))

In [57]:
first_traj_speed = np.vstack((file[500:-2501, 8:15], qt.qd))
table = np.hstack((np.arange(500, final_traj.shape[0]+500).reshape(-1, 1),final_traj, first_traj_speed))

In [58]:
# Save file
DF = pd.DataFrame(table)
DF.to_csv('traj3.csv')

In [59]:
p560.plot(final_traj, dt=0.004);

ValueError: expecting (None, 7) but got (2502, 15)

_Note: animation not working in Jupyter..._

The inverse kinematic solution was found using an iterative numerical procedure.  It is quite general but it has several drawbacks:
- it can be slow
- it may not find a solution, if the initial choice of joint coordinates is far from the solution (in the case above the default initial choice of all zeros was used)
- it may not find the solution you want, in general there are multiple solutions for inverse kinematics.  For the same end-effector pose, the robot might:
    - have it's arm on the left or right of its waist axis, 
    - the elbow could be up or down, and
    - the wrist can flipped or not flipped.  For a two-finger gripper a rotation of 
      180° about the gripper axis leaves the fingers in the same configuration.

Most industrial robots have a _spherical wrist_ which means that the last three joint axes intersect at a single point in the middle of the wrist mechanism.  We can test for this condition

In [25]:
# p560.isspherical() # not sperical

This greatly simplifies things because the last three joints only control orientation and have no effect on the end-effector position.  This means that only the first three joints define position $(x_e, y_e, z_e)$.  Three joints that control three unknowns is relatively easy to solve for, and analytical solutions (complex trigonmetric equations) can be found, and in fact have been published for most industrial robot manipulators.

The Puma560 has an analytical solution.  We can request the solution with the arm to the left and the elbow up, and the wrist not flipped by using the configuration string `"lun"`


In [26]:
#sol = p560.ikine_a(T, "lun")
#sol

which is different to the values found earlier, but we can verify it is a valid solution

In [27]:
p560.fkine(sol.q)

   1.7e-05   2.686e-05  1         0.488     
   1        -0.0002246 -1.7e-05   0.1985    
   0.0002246  1        -2.687e-05  0.4908    
   0         0         0         1         


In fact the solution we found earlier, but didn't explicitly specify, is the right-handed elbow-up configuration

In [28]:
#sol = p560.ikine_a(T, "run")
#sol.q

Other useful functions include the manipulator Jacobian which maps joint velocity to end-effector velocity expressed in the world frame

In [30]:
np.round(p560.jacob0(p560.qn),decimals=2)


array([[       0,     0.07,     0.07,     0.02,        0,        0,        0],
       [    0.09,        0,        0,        0,     0.15,        0,        0],
       [       0,    -0.09,    -0.44,    -0.22,        0,        0,        0],
       [       0,        0,        0,        0,    -0.08,        1,        0],
       [       0,        1,        1,        1,        0,        0,        1],
       [       1,        0,        0,        0,        1,     0.08,        0]])

In [31]:
np.round(p560.jacob0(p560.qn,start='link01',end='link06'),decimals=2)


array([[       0,     0.07,     0.07,     0.01,        0,        0],
       [   -0.01,        0,        0,        0,     0.05,        0],
       [       0,     0.01,    -0.34,    -0.12,        0,        0],
       [       0,        0,        0,        0,    -0.08,        1],
       [       0,        1,        1,        1,        0,        0],
       [       1,        0,        0,        0,        1,     0.08]])

In [86]:
np.round(p560.jacob0(p560.grab3),decimals=2)

AttributeError: 'Z1' object has no attribute 'grab3'